## Initialization  

In [1]:
%load_ext autoreload
%autoreload 2

import os 
import sys
sys.path.insert(0, './src')
# sys.path.insert(0, '/home/kbardool/kusanagi/AdaSparseChem/src')
# print(sys.path)
import time
import argparse
import yaml
import types
import copy, pprint
from time import sleep
from datetime import datetime

import numpy  as np
import torch  
import wandb
import pandas as pd
from utils.notebook_modules import initialize, init_dataloaders, init_environment, init_wandb, \
                                   training_prep, disp_dataloader_info,disp_info_1, disp_for_excel,\
                                   warmup_phase, weight_policy_training, disp_gpu_info

from utils.util import (print_separator, print_underline, print_yaml,  print_loss, load_from_pickle,
                        timestring, print_heading, print_dbg, get_command_line_args ) 

pp = pprint.PrettyPrinter(indent=4)
np.set_printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=6, linewidth=132)
# torch.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, profile=None, sci_mode=None)
pd.options.display.width = 132

os.environ["WANDB_NOTEBOOK_NAME"] = "Adashare_Training.ipynb"

disp_gpu_info() 

 Cuda is available  :  True
 CUDA device count  :  1
 CUDA current device:  0
 GPU Processes      : 
 GPU:0
process          4 uses        0.000 MB GPU memory

 Device : cuda:0
   name:        NVIDIA GeForce GTX 970M
   capability:  (5, 2)
   properties:  _CudaDeviceProperties(name='NVIDIA GeForce GTX 970M', major=5, minor=2, total_memory=3071MB, multi_processor_count=10)
   Allocated :  0
   Reserved  :  0

| ID | GPU  | MEM |
-------------------
|  0 | nan% |  1% |


## Create Environment

### Parse Input Args  - Read YAML config file

In [44]:
RESUME_MODEL_CKPT = 'model_train_ep_25_seed_0088'

In [37]:
## For RESTARTING
##
# input_args = " --config yamls/chembl_3task_train.yaml " \
#              " --resume " \
#              " --exp_id      330i85cg" \
#              " --exp_name    0308_1204" \
#              " --exp_desc    Train with dropout 0.5" \
#              " --seed_idx    0 "\
#              " --batch_size  128" \
#              " --lambda_sparsity  0.01"\
#              " --lambda_sharing   0.01" 
## get command line arguments

In [2]:
##  For Initiating 
##
input_args = " --config yamls/chembl_3task_train.yaml " \
             " --exp_desc    Train 6x75 layers, with dropout 0.5" \
             " --seed_idx            0" \
             " --batch_size        128" \
             " --lambda_sparsity  0.01" \
             " --lambda_sharing   0.01" 

                         

In [3]:
opt, ns = initialize(input_args)


  command line parms : 
------------------------
 config...................  yamls/chembl_3task_train.yaml
 exp_id...................  None
 exp_name.................  None
 folder_sfx...............  None
 exp_desc.................  Train 6x75 layers, with dropout 0.5
 seed_idx.................  0
 batch_size...............  128
 backbone_lr..............  None
 task_lr..................  None
 policy_lr................  None
 decay_lr_rate............  None
 decay_lr_freq............  None
 lambda_sparsity..........  0.01
 lambda_sharing...........  0.01
 gpu_ids..................  [0]
 resume...................  False
 cpu......................  False



##################################################
################### READ YAML ####################
##################################################


 log_dir              create folder:  ../experiments/AdaSparseChem/75x6_0308_2033_plr0.01_sp0.01_sh0.01
 result_dir           folder exists:  ../experiments/AdaSparseChem/75x6_03

### Setup Dataloader and Model  

In [16]:
dldrs = init_dataloaders(opt)

environ = init_environment(ns, opt, is_train = True, policy_learning = False, display_cfg = False)

# ********************************************************************
# **************** define optimizer and schedulers *******************
# ********************************************************************                                
environ.define_optimizer(policy_learning=True)
environ.define_scheduler(policy_learning=True)

##################################################
############### CREATE DATALOADERS ###############
##################################################
##################################################
############# CREATE THE ENVIRONMENT #############
##################################################
 device is  cuda:0
--------------------------------------------------------
* SparseChemEnv_Dev environment successfully created
-------------------------------------------------------- 



In [17]:
disp_dataloader_info(dldrs)


 trainset.y_class                                   :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset1.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 trainset2.y_class                                  :  [(13331, 5), (13331, 5), (13331, 5)] 
 valset.y_class                                     :  [(4137, 5), (4137, 5), (4137, 5)]  
 testset.y_class                                    :  [(920, 5), (920, 5), (920, 5)]  
                                 
 size of training set 0 (warm up)                   :  13331 
 size of training set 1 (network parms)             :  13331 
 size of training set 2 (policy weights)            :  13331 
 size of validation set                             :  4137 
 size of test set                                   :  920 
                               Total                :  45050 
                                 
 lenght (# batches) in training 0 (warm up)         :  105 
 lenght (# batches) in training 1 (network p

In [6]:
print(ns.__dict__.keys())
print(dldrs.__dict__.keys())

dict_keys(['args', 'config_filename'])
dict_keys(['trainset0', 'trainset1', 'trainset2', 'valset', 'testset', 'warmup_trn_loader', 'weight_trn_loader', 'policy_trn_loader', 'val_loader', 'test_loader'])


###  Weights and Biases Initialization 

In [7]:
init_wandb(ns, opt, environment = environ)

print(f" PROJECT NAME: {ns.wandb_run.project}\n"
      f" RUN ID      : {ns.wandb_run.id} \n"
      f" RUN NAME    : {ns.wandb_run.name}") 

wandb: Currently logged in as: kbardool (use `wandb login --relogin` to force relogin)


604b0kzu 0308_2033 AdaSparseChem


 PROJECT NAME: AdaSparseChem
 RUN ID      : 604b0kzu 
 RUN NAME    : 0308_2033
 PROJECT NAME: AdaSparseChem
 RUN ID      : 604b0kzu 
 RUN NAME    : 0308_2033


In [80]:
print(ns.__dict__.keys())
print(ns.wandb_run.project)
print(ns.wandb_run)

dict_keys(['args', 'config_filename', 'wandb_run', 'print_freq', 'eval_iters', 'stop_iter_w', 'stop_iter_a', 'flag', 'current_epoch', 'current_iter', 'best_results', 'best_metrics', 'best_value', 'best_iter', 'p_epoch', 'w_epoch', 'num_train_layers', 'leave', 'flag_warmup', 'num_prints', 'num_blocks', 'warm_up_epochs', 'training_epochs', 'curriculum_speed', 'stop_epoch_warmup', 'stop_epoch_training'])
AdaSparseChem


In [12]:
# ns.wandb_run.finish()

### Initiate / Resume Training Prep

In [8]:
print(opt['train']['resume'])
print(opt['train']['which_iter'])
print(opt['paths']['checkpoint_dir'])
print(RESUME_MODEL_CKPT)

False
warmup
../experiments/AdaSparseChem/75x6_0308_2033_plr0.01_sp0.01_sh0.01


NameError: name 'RESUME_MODEL_CKPT' is not defined

In [ ]:
# opt['train']['resume'] = True
# opt['train']['which_iter'] = 'warmup_ep_40_seed_0088'
if opt['train']['resume']:
    print_separator('Resume training')
    loaded_iter, loaded_epoch = environ.load_checkpoint(RESUME_MODEL_CKPT, path = opt['paths']['checkpoint_dir'], verbose = True)
    print(loaded_iter, loaded_epoch)    
#     current_iter = environ.load_checkpoint(opt['train']['which_iter'])
    environ.networks['mtl-net'].reset_logits()
else:
    print_separator('Initiate Training ')

In [96]:
val_metrics = load_from_pickle(opt['paths']['checkpoint_dir'], 'metrics_train_ep_25_seed_0088.pickle')

In [10]:
# training_prep(ns, opt, environ, dldrs, epoch = loaded_epoch, iter = loaded_iter )


### Training Preparation

In [18]:
training_prep(ns, opt, environ, dldrs )


 cuda available [0]
 set print_freq to length of train loader: 105
 set eval_iters to length of val loader  : 33


## Training Preparation

In [19]:
disp_info_1(ns, opt, environ)


 Num_blocks                : 6                                

 # batches in Weight training epoch : 105 
 # batches in Policy training epoch : 105 
 batch size                : 128 
 Total iterations          : 25000 
 Warm-up iterations        : None 
 Warm-up epochs            : 15 
 Warm-up stop              : 15 
 training_epochs           : 50                                 

 Print Frequency           : -1 
 Config Val Frequency      : 500 
 Config Val Iterations     : -1 
 Val iterations            : 33 
 which_iter                : warmup 
 train_resume              : False                                 
                                 
 
 fix BN parms              : False 
 Backbone LR               : 0.001 
 Backbone LR               : 0.001                                 

 Sharing  regularization   : 0.01 
 Sparsity regularization   : 0.01 
 Task     regularization   : 1 
 Last Epoch                : 0  
 # of warm-up epochs to do : 15 
 stop_iter_w               : 

In [20]:
disp_for_excel(opt)


 folder: 75x6_0308_2033_plr0.01_sp0.01_sh0.01 
 layers: [75, 75, 75, 75, 75, 75]                                
 
 middle dropout         : 0.5 
 last dropout           : 0.5 
 diff_sparsity_weights  : False 
 skip_layer             : 0 
 is_curriculum          : False 
 curriculum_speed       : 3                               
 
 decay_lr_rate          : 0.75 
 decay_lr_freq          : 2000 
 policy_decay_lr_rate   : 0.75 
 policy_decay_lr_freq   : 2200                               
 
 policy_lr              : 0.01 
 lambda_sparsity        : 0.01 
 lambda_sharing         : 0.01                               
 
 lambda_tasks           : 1 
 init_temp              : 4 
 decay_temp             : 0.965 
 decay_temp_freq        : 2 
 init_method            : random 
 init_neg_logits        : None 
 hard_sampling          : False 
 Warm-up epochs         : 15 
 training epochs        : 50


## Warmup Training

In [66]:
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])

In [21]:
print(ns.stop_epoch_warmup, ns.current_epoch)

15 0


In [26]:
print_heading(f" Last Epoch: {ns.current_epoch}   # of warm-up epochs to do:  {ns.warm_up_epochs} - Run epochs {ns.current_epoch+1} to {ns.stop_epoch_warmup}", verbose = True)

--------------------------------------------------------------------------
 Last Epoch: 15   # of warm-up epochs to do:  15 - Run epochs 16 to 15
-------------------------------------------------------------------------- 



In [30]:
warmup_phase(ns,opt, environ, dldrs, epochs = 25)

--------------------------------------------------------------------------
 Last Epoch: 50   # of warm-up epochs to do:  15 - Run epochs 51 to 75
-------------------------------------------------------------------------- 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   51 |   5.63e-04   5.63e-04   1.00e-02  4.000e+00 |    5.9647   2.0796e+00   1.6944e-02    8.0613 |   0.56962   0.77231   0.77538   0.77220 |    8.5503   2.0796e+00   1.6944e-02    10.6469 |  19.8 |
   52 |   5.63e-04   5.63e-04   1.00e-02  4.000e+00 |    5.7734   2.0796e+00   1.6944e-02    7.8699 |   0.56993   0.77311   0.77620   0.77299 |    8.5583   2.0796e+00   1.6944e-02    10.6549 |  24.5 |
   53 |   5.63e-04   5.63e-04   1.00e-02  4.000e+00 |    6.1735   2.0796e+00   1.6944e-02    8.2700 |   0.56769   0.77449   0.77765   0.77438 |    8.5207   2.0796e+00   1.694

In [119]:
# environ.save_checkpoint('warmup', current_iter, current_epoch)   
# print_loss(environ.val_metrics, f"\n[e] Warmup epoch:{current_epoch}    iter:{current_iter}")
# environ.display_trained_policy(ns.current_epoch)
# environ.log_file.flush()

In [31]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")
print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01

 Sparsity regularization     : 0.01
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 4.0000         
 Gumbel Temp decay           : 2


In [18]:
# environ.opt['train']['policy_lr']       = 0.01
# opt['train']['policy_lr']       = 0.01
# environ.opt['train']['lambda_sparsity'] = 0.1
# environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# environ.opt['train']['decay_temp_freq'] = 2
# print(environ.optimizers['alphas'].param_groups)
# print(environ.optimizers['alphas'].param_groups[0]['initial_lr'],environ.optimizers['alphas'].param_groups[0]['lr'],)
# print(environ.optimizers['weights'].param_groups[0]['initial_lr'], environ.optimizers['weights'].param_groups[0]['lr'])
# print(environ.optimizers['weights'].param_groups[1]['initial_lr'], environ.optimizers['weights'].param_groups[1]['lr'])

## Weight & Policy Training

### Training Preparation

In [32]:
if ns.flag_warmup:
    print_heading( f"** {timestring()} \n"
                   f"** Training epoch: {ns.current_epoch} iter: {ns.current_iter}   flag: {ns.flag} \n"
                   f"** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)\n"
                   f"** Switch from Warm Up training to Alternate training Weights & Policy \n"
                   f"** Take checkpoint and block gradient flow through Policy net", verbose=True)
    environ.define_optimizer(policy_learning=True)
    environ.define_scheduler(policy_learning=True)
    ns.flag_warmup = False
    ns.flag = 'update_w'
    environ.fix_alpha()
    environ.free_weights(opt['fix_BN'])

------------------------------------------------------------------------------------------------------------------------
** 2022-03-08 21:42:49:118434 
** Training epoch: 75 iter: 7875   flag: update_w 
** Set optimizer and scheduler to policy_learning = True (Switch weight optimizer from ADAM to SGD)
** Switch from Warm Up training to Alternate training Weights & Policy 
** Take checkpoint and block gradient flow through Policy net
------------------------------------------------------------------------------------------------------------------------ 



In [33]:
# current_epoch = 40
print(ns.current_epoch,ns.training_epochs)
 

75 50


In [88]:
# num_train_layers = None 
# environ.opt['is_curriculum'] = True
# environ.opt['curriculum_speed'] = 4
ns.num_train_layers = None

In [89]:
# train_total_epochs = 10
# stop_epoch_training = current_epoch +train_total_epochs

In [34]:
print(f"current_epoch           : {ns.current_epoch}") 
print(f"current_iters           : {ns.current_iter}")  
print(f"training_epochs         : {ns.training_epochs}") 
print(f"stop_epoch_training     : {ns.stop_epoch_training}")
print(f"Batches in weight epoch : {ns.stop_iter_w}")
print(f"Batches in policy epoch : {ns.stop_iter_a}")
print()

current_epoch           : 75
current_iters           : 7875
training_epochs         : 50
stop_epoch_training     : 50
Batches in weight epoch : 105
Batches in policy epoch : 105



In [97]:

print_loss(val_metrics, title = f"[e] Last epoch:{ns.current_epoch}  it:{ns.current_iter}")
# environ.display_trained_policy(ns.current_epoch)
# environ.display_trained_logits(ns.current_epoch)

[e] Last epoch:25  it:3675 -  Total Loss: 10.9495     
Task: 8.7791   Sparsity: 2.13751e+00    Sharing: 3.28845e-02 


In [35]:
print_heading(f" Last Epoch Completed : {ns.current_epoch}        # of epochs to run:  {ns.training_epochs} -->  epochs {ns.current_epoch+1} to {ns.stop_epoch_training}"
              f"\n policy_learning rate : {environ.opt['train']['policy_lr']} "
              f"\n lambda_sparsity      : {environ.opt['train']['lambda_sparsity']}"
              f"\n lambda_sharing       : {environ.opt['train']['lambda_sharing']}"
              f"\n curriculum training  : {opt['is_curriculum']}     cirriculum speed: {opt['curriculum_speed']}     num_training_layers : {ns.num_train_layers}", 
              verbose = True)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 75        # of epochs to run:  50 -->  epochs 76 to 50
 policy_learning rate : 0.01 
 lambda_sparsity      : 0.01
 lambda_sharing       : 0.01
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 



### Weight/Policy Training

In [ ]:
weight_policy_training(ns, opt, environ, dldrs, epochs = 50)

------------------------------------------------------------------------------------------------------------------------
 Last Epoch Completed : 126   # of epochs to run:  50 -->  epochs 127 to 176 
 policy_learning rate : 0.01      
 lambda_sparsity      : 0.01
 lambda_sharing       : 0.01 
 curriculum training  : False     cirriculum speed: 3     num_training_layers : None
------------------------------------------------------------------------------------------------------------------------ 

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
  127 |   5.63e-04   5.63e-04   5.62e-03  3.595e+00 |    3.2419   2.1972e+00   1.7912e-02    5.4570 |   0.59100   0.80865   0.81159   0.80855 |    8.8771   2.1972e+00   1.7912e-02    11.0922 |  23.5 |
  127 |   5.63e-04   5.63e-04   5.62e-03  3.595e+00 |    3.2658   2.2563e+00   1.5894e-02    5.53

  140 |   4.22e-04   4.22e-04   4.22e-03  3.595e+00 |    3.3271   2.3469e+00   1.2307e-02    5.6863 |   0.59777   0.81254   0.81364   0.81244 |    8.9683   2.3469e+00   1.2307e-02    11.3275 |  28.3 |
  140 |   4.22e-04   4.22e-04   4.22e-03  3.595e+00 |    3.0747   2.3437e+00   1.1073e-02    5.4295 |   0.60461   0.81115   0.81265   0.81106 |    9.0946   2.3423e+00   1.3120e-02    11.4499 |  32.9 |
 decay gumbel softmax to 3.4687200024999996

[e] Policy training epoch:140  it:21420 -  Total Loss: 11.4499     
Task: 9.0946   Sparsity: 2.34225e+00    Sharing: 1.31196e-02 

 epch: 140   softmax      s        softmax       s        softmax       s
 -----  ----------------- -    ----------------- -    ----------------- - 
   1    0.1857    0.8143  0    0.0786    0.9214  0    0.2167    0.7833  0
   2    0.6883    0.3117  1    0.6650    0.3350  1    0.9396    0.0604  1
   3    0.8456    0.1544  1    0.3333    0.6667  0    0.4184    0.5816  0
   4    0.4365    0.5635  0    0.3865    0.6135  0 

  153 |   3.16e-04   3.16e-04   4.22e-03  3.469e+00 |    2.3450   2.4053e+00   1.3947e-02    4.7642 |   0.60342   0.81169   0.81476   0.81160 |    8.9988   2.4053e+00   1.3947e-02    11.4180 |  26.0 |
  153 |   3.16e-04   3.16e-04   4.22e-03  3.469e+00 |    2.4117   2.3855e+00   1.3660e-02    4.8108 |   0.60073   0.81314   0.81490   0.81305 |    9.0198   2.3857e+00   7.0861e-03    11.4125 |  28.7 |
  154 |   3.16e-04   3.16e-04   4.22e-03  3.469e+00 |    2.4460   2.3857e+00   7.0861e-03    4.8388 |   0.61677   0.81256   0.81402   0.81245 |    9.2582   2.3857e+00   7.0861e-03    11.6510 |  28.2 |
  154 |   3.16e-04   3.16e-04   4.22e-03  3.469e+00 |    2.9269   2.3745e+00   2.0131e-02    5.3215 |   0.60272   0.81392   0.81567   0.81382 |    9.0306   2.3743e+00   2.0202e-02    11.4251 |  31.7 |
  155 |   3.16e-04   3.16e-04   4.22e-03  3.469e+00 |    2.5066   2.3743e+00   2.0202e-02    4.9012 |   0.61686   0.81406   0.81571   0.81396 |    9.2183   2.3743e+00   2.0202e-02    11.6128 |  28

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
  166 |   3.16e-04   3.16e-04   3.16e-03  3.347e+00 |    2.8646   2.3760e+00   8.2993e-03    5.2489 |   0.60197   0.81546   0.81683   0.81535 |    9.0439   2.3760e+00   8.2993e-03    11.4283 |  25.2 |
  166 |   3.16e-04   3.16e-04   3.16e-03  3.347e+00 |    2.7011   2.3981e+00   1.0274e-02    5.1094 |   0.61265   0.81309   0.81579   0.81299 |    9.1676   2.3978e+00   7.6084e-03    11.5730 |  33.6 |
  167 |   3.16e-04   3.16e-04   3.16e-03  3.347e+00 |    2.3906   2.3978e+00   7.6084e-03    4.7960 |   0.60642   0.81394   0.81572   0.81383 |    9.0926   2.3978e+00   7.6084e-03    11.4981 |  28.1 |
  167 |   3.16e-04   3.16e-04   3.16e-03  3.347e+00 |    3.2740   2.4281e+00   9.1150e-03    5.7112 |   0.61538   0.81359   0.81532   0.81349 |    9.2309   2.4276e+00   1.0712e-02    11.6692 |  30

In [37]:
opt['train']['decay_temp_freq'] = 16
ns.best_epoch = 0


In [102]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.01

 Sparsity regularization     : 0.01
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 2.3441         
 Gumbel Temp decay           : 8


In [89]:
environ.opt['train']['policy_lr']       = 0.002
environ.opt['train']['lambda_sparsity'] = 0.1
environ.opt['train']['lambda_sharing']  = 0.01
# environ.opt['train']['lambda_tasks']    = 1.0
# # environ.opt['train']['decay_temp_freq'] = 2
 

In [90]:
print( f" Backbone Learning Rate      : {environ.opt['train']['backbone_lr']}\n"
       f" Tasks    Learning Rate      : {environ.opt['train']['task_lr']}\n"
       f" Policy   Learning Rate      : {environ.opt['train']['policy_lr']}\n")


print( f" Sparsity regularization     : {environ.opt['train']['lambda_sparsity']}\n"
       f" Sharing  regularization     : {environ.opt['train']['lambda_sharing']} \n\n"
       f" Tasks    regularization     : {environ.opt['train']['lambda_tasks']}   \n"
       f" Gumbel Temp                 : {environ.gumbel_temperature:.4f}         \n" #
       f" Gumbel Temp decay           : {environ.opt['train']['decay_temp_freq']}") #

print()
print( f" current_iters               : {current_iter}")  
print( f" current_epochs              : {current_epoch}") 
print( f" train_total_epochs          : {train_total_epochs}") 
print( f" stop_epoch_training         : {stop_epoch_training}")

 Backbone Learning Rate      : 0.001
 Tasks    Learning Rate      : 0.001
 Policy   Learning Rate      : 0.002

 Sparsity regularization     : 0.1
 Sharing  regularization     : 0.01 

 Tasks    regularization     : 1   
 Gumbel Temp                 : 0.2764         
 Gumbel Temp decay           : 2

 current_iters               : 32550
 current_epochs              : 160
 train_total_epochs          : 10
 stop_epoch_training         : 160


In [ ]:
# pp.pprint(environ.losses)
# pp.pprint(environ.val_metrics)

In [ ]:
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch)
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [163]:
# model_label   = 'model_train_ep_%d_seed_%04d' % (current_epoch, opt['random_seed'])
# metrics_label = 'metrics_train_ep_%d_seed_%04d.pickle' % (current_epoch, opt['random_seed'])
# environ.save_checkpoint(model_label, current_iter, current_epoch) 
# save_to_pickle(environ.val_metrics, environ.opt['paths']['checkpoint_dir'], metrics_label)
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()


# environ.save_checkpoint('best_weights_policy_2', current_iter, current_epoch) 
# print_loss(environ.val_metrics, title = f"[Final] ep:{current_epoch}  it:{current_iter}",)
# environ.display_trained_policy(current_epoch,out=[sys.stdout, environ.log_file])
# environ.display_trained_logits(current_epoch)
# environ.log_file.flush()

In [ ]:
# print_loss(current_iter, environ.losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, trn_losses, title = f"[e] Policy training epoch:{current_epoch}    iter:")
# print()
# print_loss(current_iter, environ.val_metrics, title = f"[e] Policy training epoch:{current_epoch}    iter:")

In [55]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics, 0, out=[sys.stdout])

Epoch | BckBone LR   Heads LR  Policy LR Gumbl Temp |  trn loss     trn spar     trn shar   trn ttl |   bceloss  avg prec    aucroc     aucpr |  val loss     val spar     val shar    val ttl |  time |
   90 |   5.22e-04   5.22e-04   6.14e-03  9.968e-01 |    2.3488   2.3753e+00   2.9361e-02    4.7535 |   0.66047   0.80706   0.81083   0.80696 |    9.8989   2.3745e+00   3.2183e-02    12.3056 |2719.3 |


In [ ]:
# environ.losses

# environ.val_metrics

In [59]:
# environ.batch_data

# environ.display_parameters()

# with np.printoptions(edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', precision=7, formatter={'float': lambda x: f"{x:12.5e}"}):
#     environ.print_logit_grads('gradients')

# environ_params = environ.get_task_specific_parameters()
# environ_params = environ.get_arch_parameters()
# environ_params = environ.get_backbone_parameters()
# print(environ_params)
# for param in environ_params:
#     print(param.grad.shape, '\n', param.grad)
#     print(param)

In [139]:
environ.display_trained_logits(ns.current_epoch)
environ.display_trained_policy(ns.current_epoch)


 epch: 125   logits       s          logits      s         logits       s
 -----  ----------------- -    ----------------  -    ----------------  - 
   1   -0.5929    0.4841  0   -0.5923    0.6541  0   -0.5925    0.9035  0
   2    0.0389   -0.1261  1    0.0389   -0.0274  1    0.0402   -0.0994  1
   3    0.2928   -0.5825  1    0.2925   -0.3840  1    0.2930   -0.0701  1
   4    0.2900   -0.1577  1    0.2906   -0.2286  1    0.2902   -0.8290  1
   5   -0.1057    0.5628  0   -0.1070   -0.4826  1   -0.1082    0.2616  0
   6   -0.2416    0.8389  0   -0.2409    0.0414  0   -0.2395   -0.3599  1



 epch: 125   softmax      s        softmax       s        softmax       s
 -----  ----------------- -    ----------------- -    ----------------- - 
   1    0.2541    0.7459  0    0.2233    0.7767  0    0.1830    0.8170  0
   2    0.5412    0.4588  1    0.5166    0.4834  1    0.5348    0.4652  1
   3    0.7058    0.2942  1    0.6630    0.3370  1    0.5898    0.4102  1
   4    0.6101    0.3899  1    0

In [ ]:
environ.display_test_sample_policy(current_epoch, hard_sampling = True)
environ.display_train_sample_policy(current_epoch, hard_sampling = True)

In [166]:
# environ.define_optimizer(policy_learning=True)

In [103]:
print(environ.optimizers['alphas'])
print(environ.optimizers['weights'])

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    initial_lr: 0.01
    lr: 0.01
    weight_decay: 0.0005
)
SGD (
Parameter Group 0
    dampening: 0
    initial_lr: 0.001
    lr: 0.00085
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001

Parameter Group 1
    dampening: 0
    initial_lr: 0.001
    lr: 0.00085
    momentum: 0.9
    nesterov: False
    weight_decay: 0.0001
)


In [104]:
print('Policy  initial_lr : ', environ.optimizers['alphas'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['alphas'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[0]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[0]['lr'])
print('Weights initial_lr : ', environ.optimizers['weights'].param_groups[1]['initial_lr'], 'lr : ',environ.optimizers['weights'].param_groups[1]['lr'])

Policy  initial_lr :  0.01 lr :  0.01
Weights initial_lr :  0.001 lr :  0.00085
Weights initial_lr :  0.001 lr :  0.00085


### Close WandB run

In [140]:
wandb.finish()

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_time,▃▂▂▂▂▃▃▂▃▂▂▃▂▂▂▂▂▂▂▃▃▂▂▃▄▄▆▆▆▂█▃██▇▄▄▁▆▆
epoch,125
train_time,27.60705


## Post Training Stuff

In [78]:
# opt['exp_instance'] = '0218_1358'     
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"
# print()
# opt['exp_instance'] = datetime.now().strftime("%m%d_%H%M")
# opt['exp_description'] = f"No Alternating Weight/Policy - training all done with both weights and policy"
# folder_name=  f"{opt['exp_instance']}_bs{opt['train']['batch_size']:03d}_{opt['train']['decay_lr_rate']:3.2f}_{opt['train']['decay_lr_freq']}"

In [101]:
wandb.finish()

wandb: WARNING Upgrade to the 0.9.49 version of W&B Local to get the latest features. Learn more: http://wandb.me/local-upgrade


In [ ]:
# 
p = environ.get_current_state(0)

In [ ]:
pp.pprint(p)

### Post Warm-up Training stuff

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
environ.networks['mtl-net'].arch_parameters()

In [ ]:
p = environ.get_sample_policy(hard_sampling = False)
print(p)
p = environ.get_policy_prob()
print(p)
p = environ.get_policy_logits()
print(p)

# p = environ.get_current_policy()
# print(p)

In [ ]:
a = softmax([0.0, 1])
print(a)
sampled = np.random.choice((1, 0), p=a)
print(sampled)

In [ ]:
print(environ.optimizers['weights'])
print(environ.schedulers['weights'].get_last_lr())

In [ ]:
print('losses.keys      : ', environ.losses.keys())
print('losses[task]keys : ', environ.losses['task1'].keys())
pp.pprint(environ.losses)

In [ ]:
print( environ.val_metrics.keys())
# pp.pprint(val_metrics)
print(type(environ.val_metrics['aggregated']))
print()
print(type(environ.val_metrics['task1']['classification_agg']))
print()
pp.pprint(environ.val_metrics)

In [ ]:
# import pickle
# with open("val_metrics.pkl", mode= 'wb') as f:
#         pickle.dump(val_metrics, f)
    
# with open('val_metrics.pkl', 'rb') as f:    
#     tst_val_metrics = pickle.load(f)

In [ ]:
# print(environ.input.shape) 
# a = getattr(environ, 'task1_pred')
# yc_data = environ.batch['task1_data']
# print(yc_data.shape)
# yc_ind = environ.batch['task1_ind']
# print(yc_ind.shape)
# yc_hat_all = getattr(environ, 'task1_pred')
# print(yc_hat_all.shape)
# yc_hat  = yc_hat_all[yc_ind[0], yc_ind[1]]
# print(yc_hat_all.shape, yc_hat.shape)

# 
# environ.losses
# loss = {}
# for key in environ.losses.keys():
#     loss[key] = {}
#     for subkey, v in environ.losses[key].items():
#         print(f" key:  {key}   subkey: {subkey} ")
#         if isinstance(v, torch.Tensor):
#             loss[key][subkey] = v.data
#             print(f" Tensor  -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
#         else:
#             loss[key][subkey] = v
#             print(f" integer -  key:  {key}   subkey: {subkey}           value type: {type(v)}  value: {v:.4f}")
# pp.pprint(tst_val_metrics)             

In [ ]:
# print('metrics.keys: ', environ.metrics.keys())
# print('metrics[task].keys: ', environ.metrics['task1'].keys())
# pp.pprint(environ.metrics['task1'])
# pp.pprint(environ.losses['task1']['total'])

In [ ]:
# title='Iteration'
# for t_id, _ in enumerate(environ.tasks):
#     task_key = f"task{t_id+1}"
# #     print_heading(f"{title}  {current_iter}  {task_key} : {val_metrics[task_key]['classification_agg']}", verbose = True)

#     for key, _  in val_metrics[task_key]['classification_agg'].items():
#         print('%s/%-20s'%(task_key, key), val_metrics[task_key]['classification_agg'][key], current_iter)
#         print(f"{task_key:s}/{key:20s}", val_metrics[task_key]['classification_agg'][key], current_iter)
#         print()
#             # print_current_errors(os.path.join(self.log_dir, 'loss.txt'), current_iter,key, loss[key], time.time() - start_time)

In [ ]:
# environ.print_loss(current_iter, start_time, metrics = val_metrics['loss'], verbose=True)
# print(opt['lambdas'])
# p = (opt['lambdas'][0] * environ.losses['tasks']['task1'])
# print(p)

# environ.print_val_metrics(current_iter, start_time, val_metrics , title='validation', verbose=True)    

In [ ]:
# print(current_iter)
# print_metrics_cr(current_iter, t1 - t0, None, val_metrics , True)
# environ.print_val_metrics(current_iter, start_time, val_metrics, title='validation', verbose = True)

In [ ]:
print(f" val_metric keys               : {val_metrics.keys()}")
print(f" loss keys                     : {val_metrics['loss'].keys()}")
print(f" task1 keys                    : {val_metrics['task1'].keys()}")
print(f" task1 classification keys     : {val_metrics['task1']['classification'].keys()}")
print(f" task1 classification_agg keys : {val_metrics['task1']['classification_agg'].keys()}")
print()
print(f" task1                       : {val_metrics['task1']['classification_agg']['loss']:5f}")
print(f" task2                       : {val_metrics['task2']['classification_agg']['loss']:5f}")
print(f" task3                       : {val_metrics['task3']['classification_agg']['loss']:5f}")
print(f" loss                        : {val_metrics['loss']['total']:5f}")
print(f" train_time                  : {val_metrics['train_time']:2f}")
print(f" epoch                       : {val_metrics['epoch']}")


### Post Weight + Policy Training Stuff 

In [ ]:
environ.networks['mtl-net'].backbone.layer_config

In [ ]:
num_blocks = 6
num_policy_layers = 6
gt =  torch.ones((num_blocks)).long()
gt0 =  torch.zeros((num_blocks)).long()
print(gt)
print(gt0)

loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
print(loss_weights)

In [ ]:
if environ.opt['diff_sparsity_weights'] and not environ.opt['is_sharing']:
    print(' cond 1')
    ## Assign higher weights to higher layers 
    loss_weights = ((torch.arange(0, num_policy_layers, 1) + 1).float() / num_policy_layers)
    print(f"{task_key} sparsity error:  {2 * (loss_weights[-num_blocks:] * environ.cross_entropy2(logits[-num_blocks:], gt)).mean()})")
    print_dbg(f" loss_weights :  {loss_weights}", verbose = True)
    print_dbg(f" cross_entropy:  {environ.cross_entropy2(logits[-num_blocks:], gt)}  ", verbose = True)
    print_dbg(f" loss[sparsity][{task_key}]: {self.losses['sparsity'][task_key] } ", verbose = True)

else:
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between \n Logits   : \n{logits[-num_blocks:]} \n and gt: \n{gt} \n", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-num_blocks:], gt)}")
    
    print('\n cond 2')
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt[-1:])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits      : {logits[-1:]}  and gt: {gt0[-1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[-1:], gt0[-1:])} \n")
    
    print('\n cond 3')    
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt[0:1])} \n")
    print_dbg(f"Compute CrossEntropyLoss between Logits   : {logits[0:1]}  and gt: {gt0[0:1]} ", verbose = True)
    print(f"{task_key} sparsity error:  {environ.cross_entropy_sparsity(logits[0:1], gt0[0:1])} \n")
        
        

In [ ]:
# flag = 'update_w'
# environ.fix_alpha
# environ.free_w(opt['fix_BN'])

flag = 'update_alpha'
environ.fix_weights()
environ.free_alpha()

In [ ]:
environ.networks['mtl-net'].num_layers

In [ ]:
print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

train_total_epochs += 5

print(f"current_iters         : {current_iter}")  
print(f"current_epochs           : {current_epoch}") 
print(f"train_total_epochs    : {train_total_epochs}") 

In [ ]:
# print_metrics_cr(current_epoch, time.time() - t0, None, environ.val_metrics , num_prints)      

# num_prints += 1
# t0 = time.time()

# # Take check point
# environ.save_checkpoint('latest', current_iter)
# environ.train()
# #-------------------------------------------------------
# # END validation process
# #-------------------------------------------------------       
# flag = 'update_alpha'
# environ.fix_w()
# environ.free_alpha()

In [ ]:
# dilation = 2
# kernel_size = np.asarray((3, 3))
# upsampled_kernel_size = (kernel_size - 1) * (dilation - 1) + kernel_size
# print(upsampled_kernel_size)

In [ ]:
# environ.optimizers['weights'].param_groups[0]
# for param_group in optimizer.param_groups:
#     return param_group['lr']

In [ ]:
environ.schedulers['weights'].get_last_lr()

In [ ]:
current_state = {}
for k, v in environ.optimizers.items():
    print(f'state dict for {k} = {v}')
    current_state[k] = v.state_dict()
pp.pprint(current_state)

In [ ]:
current_state = {}
for k, v in environ.schedulers.items():
    print(f'state dict for {k} = {v}')
    print(v.state_dict())

### Losses and Metrics

In [ ]:
trn_losses = environ.losses

In [ ]:
print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# print_metrics_cr(current_epoch, time.time() - start_time, trn_losses, environ.val_metrics , num_prints)      

In [ ]:
# pp.pprint(environ.losses)
pp.pprint(trn_losses)

In [ ]:
pp.pprint(environ.val_metrics)

In [ ]:
# environ.opt['train']['Lambda_sharing'] = 0.5
# opt['train']['Lambda_sharing'] = 0.5

# environ.opt['train']['policy_lr'] = 0.001
# opt['train']['policy_lr'] = 0.001

In [ ]:
environ.losses.keys()
pp.pprint(environ.losses)

In [ ]:
tmp = environ.get_loss_dict()
print(tmp.keys())
pp.pprint(tmp)

In [ ]:
print(opt['diff_sparsity_weights'])
print(opt['is_sharing'])
print(opt['diff_sparsity_weights'] and not opt['is_sharing'])
print(environ.opt['train']['Lambda_sharing'])
print(opt['train']['Lambda_sharing'])
print(environ.opt['train']['Lambda_sparsity'])
print(opt['train']['Lambda_sparsity'])
print(environ.opt['train']['policy_lr'])
print(opt['train']['policy_lr'])

### Policy / Logit stuff

In [ ]:
from scipy.special          import softmax

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan')
torch.set_printoptions(precision=8,linewidth=132)

#### `get_task_logits(n)` Get logits for task group n

In [ ]:
task_logits = environ.get_task_logits(1)
print(task_logits)

#### `get_arch_parameters()`: Get last used logits from network

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

In [ ]:
import torch.optim as optim
arch_parameters      = environ.get_arch_parameters()
print(arch_parameters)

#### `get_policy_logits()`:  Get Policy Logits - returns same as `get_arch_parameters()`

In [ ]:
logs = environ.get_policy_logits()
for i in logs:
    print(i, '\n')
# probs = softmax(logs, axis= -1)
# for i in probs:
#     print(i, '\n')

#### `get_policy_prob()` : Gets the softmax of the logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
for i in policy_softmaxs:
    print(i, '\n')

#### `get_sample_policy( hard_sampling = False)` : Calls test_sample_policy of network with random choices based on softmax of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
policies,logits = environ.get_sample_policy(hard_sampling = False)

for l, p, s in zip(logits, policies, policy_softmaxs) :
    for  l_row, p_row, s_row in zip(l, p, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### `get_sample_policy( hard_sampling = True)` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
policy_softmaxs = environ.get_policy_prob()
hard_policies, logits = environ.get_sample_policy(hard_sampling = True)

for p,l,s in zip(hard_policies, logits, policy_softmaxs) :
    for  p_row, l_row, s_row in zip(p, l, s):
        print( l_row,'\t', p_row, '\t', s_row)
    print('\n')

#### Print

In [ ]:
print(f" Layer    task 1      task 2      task 3")
print(f" -----    ------      ------      ------")
for idx, (l1, l2, l3) in enumerate(zip(hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1}       {l2}       {l3}")
    

    print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
def display_trained_policy(iter):

    policy_softmaxs = environ.get_policy_prob()
    policy_argmaxs = 1-np.argmax(policy_softmaxs, axis = -1)
    print(f"  Trained polcies at iteration: {iter} ")
    print(f"                   task 1                           task 2                         task 3        ")
    print(f" Layer       softmax        select          softmax        select          softmax        select   ")
    print(f" -----    ---------------   ------       ---------------   ------       ---------------   ------   ")
    for idx, (l1,l2,l3,  p1,p2,p3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2], policy_argmaxs[0], policy_argmaxs[1], policy_argmaxs[2]),1):
        print(f"   {idx}      {l1[0]:.4f}   {l1[1]:.4f}   {p1:4d}    {l2[0]:11.4f}   {l2[1]:.4f}   {p2:4d}    {l3[0]:11.4f}   {l3[1]:.4f}   {p3:4d}")

    print()
# print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
display_trained_policy(5)

In [ ]:
print(f"                        POLICIES (SOFTMAX)                                       task 3          ")
print(f" Layer    task1              task2            task3 softmax         softmax         argmax         softmax         argmax   ")
print(f" -----    -------------     -------------     -------------   ------   ")
for idx, (l1,l2,l3, h1,h2,h3) in enumerate(zip(policy_softmaxs[0], policy_softmaxs[1], policy_softmaxs[2],hard_policies[0], hard_policies[1], hard_policies[2]),1):
    print(f"   {idx}      {l1[0]:.4f} {l1[1]:.4f}     {l2[0]:.4f} {l2[1]:.4f}     {l3[0]:.4f} {l3[1]:.4f}    {h3}")
    
print(f"\n\n where [p1  p2]:  p1: layer is selected    p2: layer is not selected")

In [ ]:
# print(policy_softmaxs[2], np.argmax(1-policy_softmaxs[2], axis = -1))
print(policy_softmaxs, np.argmax(policy_softmaxs, axis = -1))

#### `get_current_logits()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
logits  = (environ.get_current_logits())
for i in logits:
    print(i ,'\n')

#### `get_current_policy()` : Calls test_sample_policy of network using ARGMAX of logits

In [ ]:
pols  = (environ.get_current_policy())

for i in pols:
    print(i ,'\n')

#### `gumbel_softmax()`  

In [ ]:
np.set_printoptions(precision=8,edgeitems=3, infstr='inf', linewidth=150, nanstr='nan', floatmode = 'maxprec_equal')
torch.set_printoptions(precision=8,linewidth=132)

In [ ]:
print(environ.temp)
# tau = environ.temp
tau = 1
for i in range(3): 
    logits_tensor = torch.tensor(logits[0])
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=tau, hard=False).cpu().numpy() 

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=tau, hard=True).cpu().numpy()
    
    for l, gs, gh in zip(lgts, gumbel_soft, gumbel_hard):
        print(f"   {l}   \t {gs}            \t {gh}")
#     print(lgts)
#     print(gumbel_soft)
#     print(gumbel_hard)
    print()

In [ ]:
for lgts in logits:
    logits_tensor = torch.tensor(lgts)
    print(lgts)
    # Sample soft categorical using reparametrization trick:
    gumbel_soft = F.gumbel_softmax(logits_tensor, tau=1, hard=False)
    print(gumbel_soft)

    # Sample hard categorical using "Straight-through" trick:
    gumbel_hard  = F.gumbel_softmax(logits_tensor, tau=1, hard=True)
    print(gumbel_hard)
    print()

In [ ]:
smax = scipy.special.softmax(logs, axis =1)
# smax = np.array( 
# [[0.46973792, 0.530262  ],
#  [0.45025694, 0.549743  ],
#  [0.4443086 , 0.5556915 ],
#  [0.4138397 , 0.58616036],
#  [0.4140113 , 0.5859887 ],
#  [0.42114905, 0.57885087]])

print(smax.shape)
print(smax)
print(smax[0])
print(smax[0].sum())
print(np.random.choice((1,0), p =smax[0]))

In [ ]:
logs = np.array(
[[0.33064184, 0.42053092],
 [0.3532089 , 0.52056104],
 [0.3888512 , 0.5680909 ],
 [0.42039296, 0.694217  ],
 [0.4519742 , 0.73311865],
 [0.48401102, 0.7522658 ]],
)